# Holoviews / Bokeh Example of Images and Regions

Necessary conda installations: `holoviews, bokeh, ipywidgets, astropy

In [1]:
import holoviews as hv
from holoviews import opts, streams
import ipywidgets
hv.extension('bokeh')

ModuleNotFoundError: No module named 'holoviews'

## Example from the Internet

In [2]:
path = hv.Path([[(1, 5), (9, 5)]])
poly = hv.Polygons([[(2, 2), (5, 8), (8, 2)]])
path_stream = streams.PolyDraw(source=path, drag=True, show_vertices=True)
poly_stream = streams.PolyDraw(source=poly, drag=True, num_objects=2, show_vertices=True)

(path * poly).opts(
    opts.Path(color='red', height=400, line_width=5, width=400),
    opts.Polygons(fill_alpha=0.3, active_tools=['poly_draw']))

:Overlay
   .Path.I     :Path   [x,y]
   .Polygons.I :Polygons   [x,y]

In [3]:
poly_stream

PolyDraw(data={'xs': [array([2, 5, 8])], 'ys': [array([2, 8, 2])]})

## Load FITS file and display slice as an Image

In [4]:
from astropy.io import fits

# Load the data
f = fits.open('/home/craig/Documents/STScI/cubeviz/data/manga-7495-12704-LOGCUBE.fits.gz')
image_data = f[1].data

In [5]:
img = hv.Image(image_data[10])
img


:Image   [x,y]   (z)

## Display Image along with FITS extension selection and slice

In [14]:
# Callback
def image(ext, slice):
    return hv.Image(f[ext].data[int(slice)])

# Region selection
poly = hv.Polygons([[(0,0), (0, 0)]])
poly_stream = streams.PolyDraw(source=poly, drag=True, num_objects=2, show_vertices=True)

# Display
dmap = hv.DynamicMap(image, kdims=['ext', 'slice'])
(dmap.redim.range(slice=(0, image_data.shape[0]))* poly).redim.values(ext=('FLUX', 'IVAR', 'MASK')).opts(
    opts.Polygons(fill_alpha=0.3, active_tools=['poly_draw']))

:DynamicMap   [ext,slice]
   :Overlay
      .Image.I    :Image   [x,y]   (z)
      .Polygons.I :Polygons   [x,y]

In [18]:
# poly_stream will be updated with each change in regions
poly_stream

poly_stream.data['xs'][0] = [-0.1, 0.1]
poly_stream.data['ys'][0] = [-0.1, -0.1]

poly_stream

PolyDraw(data={'xs': [[-0.1, 0.1]], 'ys': [[-0.1, -0.1]]})